In [1]:
import numpy as np
import copy
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [2]:
import torch
import torch.nn as nn
import dtnnlib as dtnn
# import resnet_cifar

from torchvision import datasets, transforms as T
from torch.utils import data

In [3]:
from tqdm import tqdm
import os, time, sys
import json

In [4]:
mnist_transform = T.Compose([
    T.ToTensor(),
    T.Normalize(
        mean=[0.5,],
        std=[0.5,],
    ),
])

train_dataset = datasets.FashionMNIST(root="../../../../_Datasets/", train=True, download=True, transform=mnist_transform)
test_dataset = datasets.FashionMNIST(root="../../../../_Datasets/", train=False, download=True, transform=mnist_transform)

In [5]:
# train_dataset.data = train_dataset.data.view(-1, 28*28)
# test_dataset.data = test_dataset.data.view(-1, 28*28)

In [6]:
batch_size = 50
train_loader = data.DataLoader(dataset=train_dataset, num_workers=4, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, num_workers=4, batch_size=batch_size, shuffle=False)

In [7]:
device = torch.device("cuda:1")

In [8]:
criterion = nn.CrossEntropyLoss()

In [9]:
for xx, yy in train_loader:
    xx, yy = xx.to(device), yy.to(device)
    print(xx.shape, yy.shape)
    break

torch.Size([50, 1, 28, 28]) torch.Size([50])


## Any function as metric

In [10]:
class FunctionDT(nn.Module):
    
    def __init__(self, input_dim, num_centers, func, inv_temp=0.):
        '''
        func [input_dim -> 1]
        '''
        super().__init__()
        self.input_dim = input_dim
        self.num_centers = num_centers
        self.func = func
        
        self.inv_temp = nn.Parameter(torch.ones(1)*inv_temp)
        
        self.centers = torch.randn(num_centers, input_dim)/3.
        self.centers = nn.Parameter(self.centers)
    
    def forward(self, x):
        z = x.unsqueeze(1) - self.centers.unsqueeze(0)
        dists = self.func(z).squeeze(-1)
        dists = -dists*torch.exp(self.inv_temp)
        
        ## COmment out (un-normalized)
#         dists = dists-dists.mean(dim=1, keepdim=True)
#         dists = dists/dists.std(dim=1, keepdim=True)

        return dists

In [11]:
from classes import DistanceRegressor, ConvexNN
from nflib.flows import SequentialFlow, ActNorm
import nflib.res_flow as irf

## Merge all models into single and benchmark

In [29]:
def get_models(h = 5, device='cpu'):
    I = 784
    layer1 = {
#         "l_0.5":dtnn.DistanceTransform(I, h, p=0.5, bias=False),
        "l_1":dtnn.DistanceTransform(I, h, p=1, bias=False),
        "l_2":dtnn.DistanceTransform(I, h, bias=False),
#         "l_inf":dtnn.DistanceTransform(I, h, p=20, bias=False),
#         "stereo":dtnn.StereographicTransform(I, h, bias=False),
        "linear":nn.Linear(I, h, bias=False)
    }
    net_dict = {}
    for key in layer1:
        net = nn.Sequential(
            layer1[key],
#             nn.BatchNorm1d(h),
            nn.LayerNorm(h),
            nn.ELU(),
            nn.Linear(h, 10),
            )
        net_dict[key] = net.to(device)
    return net_dict

In [30]:
def get_models_func(h = 500, func_h=500, device='cpu'):
    layer1 = {
        "convex":ConvexNN([784, func_h, 1]),
        "invex":nn.Sequential(
                    ActNorm(784),
                    irf.ResidualFlow(784, [func_h], activation=irf.LeakyReLU),
                    ActNorm(784),
                    DistanceRegressor(784),
                    ),
        "ordinary":nn.Sequential(
                    ActNorm(784),
                    irf.ResidualFlow(784, [func_h], activation=irf.LeakyReLU),
                    ActNorm(784),
#                     DistanceRegressor(784),
                    nn.Linear(784, 1),
                    ),
    }
    irf.remove_spectral_norm_model(layer1["ordinary"])

    net_dict = {}
    for key in layer1:
        net = nn.Sequential(
            FunctionDT(784, h, layer1[key]),
            nn.BatchNorm1d(h),
            nn.ELU(),
            nn.Linear(h, 10),
            )
        net_dict[key] = net.to(device)
    return net_dict

In [31]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch, model, optimizer):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device).view(-1, 28*28), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    return

In [32]:
best_acc = -1
def test(epoch, model, model_name):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device).view(-1, 28*28), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/{model_name}.pth')
        best_acc = acc

In [33]:
learning_rate = 0.0001
EPOCHS = 50

In [34]:
exp_acc_vals = {}

In [19]:
data_file = "./outputs/00.2_exp_acc_dict_bkp0(un-normalized).json"

with open(data_file, 'r') as f:
    exp_acc_vals = json.load(f)

In [35]:
exp_acc_vals

{}

In [36]:
H = [5, 10, 20, 100, 500]
# net_keys = {**get_models(1), **get_models_func(1)}.keys()
net_keys = {**get_models(1)}.keys()
# net_keys = {**get_models_func(1)}.keys()

net_keys

dict_keys(['l_1', 'l_2', 'linear'])

In [37]:
# H_ = [h for h in H if str(h) not in exp_acc_vals.keys()]
H_ = []
for h in H:
    if str(h) in exp_acc_vals.keys():
        NET_KEYS = [k for k in net_keys if k not in list(exp_acc_vals[str(h)].keys())]
        print(h, NET_KEYS)
        if len(NET_KEYS) > 0:
            H_.append(h)
    else:
        H_.append(h)

H_

[5, 10, 20, 100, 500]

In [38]:
# h = 20
# NET_KEYS = list(net_keys)
# if str(h) in exp_acc_vals.keys():
#     NET_KEYS = [k for k in net_keys if k not in list(exp_acc_vals[str(h)].keys())[:-2]]

In [39]:
# NET_KEYS

In [40]:
# str(h) in exp_acc_vals.keys()

In [41]:
for h in H_:
    net_dict = {**get_models(h)}
#     net_dict = {**get_models(h), **get_models_func(h)}
    acc_dict = {}
    NET_KEYS = list(net_keys)
    
    if str(h) in exp_acc_vals.keys():
        acc_dict = exp_acc_vals[str(h)]
        NET_KEYS = [k for k in net_keys if k not in list(exp_acc_vals[str(h)].keys())]
        pass
                    
    for key in NET_KEYS:
        print("_________________________")
        print(f"Experimenting for {key} ; h:{h}")
        net = net_dict[key].to(device)
        
        model_name = f"00.2_fmnist_{key}_h{h}"
        
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)
        best_acc = -1
        for epoch in range(EPOCHS):
            train(epoch, net, optimizer)
            test(epoch, net, model_name)
            scheduler.step()
        acc_dict[key] = float(best_acc)
        exp_acc_vals[str(h)] = acc_dict
        
        ## Save it in the file.
#         with open(data_file, "w") as f:
#             json.dump(exp_acc_vals, f, indent=3)
        
        pass
        

_________________________
Experimenting for l_1 ; h:5


100%|████████████████████████████████████████████████| 1200/1200 [00:02<00:00, 522.50it/s]


[Train] 0 Loss: 2.096 | Acc: 25.952 15571/60000


100%|██████████████████████████████████████████████████| 200/200 [00:00<00:00, 400.45it/s]


[Test] 0 Loss: 1.902 | Acc: 37.510 3751/10000
Saving..


...

[Test] 48 Loss: 0.305 | Acc: 89.340 8934/10000


100%|████████████████████████████████████████████████| 1200/1200 [00:02<00:00, 547.92it/s]


[Train] 49 Loss: 0.144 | Acc: 95.265 57159/60000


100%|██████████████████████████████████████████████████| 200/200 [00:00<00:00, 413.16it/s]

[Test] 49 Loss: 0.305 | Acc: 89.320 8932/10000


In [42]:
exp_acc_vals

{'5': {'l_1': 76.93, 'l_2': 77.87, 'linear': 81.36},
 '10': {'l_1': 81.27, 'l_2': 82.57, 'linear': 85.35},
 '20': {'l_1': 82.68, 'l_2': 83.23, 'linear': 86.25},
 '100': {'l_1': 85.53, 'l_2': 86.22, 'linear': 88.5},
 '500': {'l_1': 86.82, 'l_2': 87.6, 'linear': 89.46}}

In [26]:
exp_acc_vals

{'5': {'l_0.5': 67.05,
  'l_1': 70.7,
  'l_2': 78.15,
  'l_inf': 79.52,
  'stereo': 82.19,
  'linear': 82.74,
  'convex': 79.49,
  'invex': 88.26,
  'ordinary': 83.26},
 '10': {'l_0.5': 72.08,
  'l_1': 77.91,
  'l_2': 82.35,
  'l_inf': 83.98,
  'stereo': 84.73,
  'linear': 84.89,
  'convex': 78.99,
  'invex': 88.41,
  'ordinary': 86.11},
 '20': {'l_0.5': 75.54,
  'l_1': 80.08,
  'l_2': 83.82,
  'l_inf': 85.17,
  'stereo': 85.86,
  'linear': 86.27,
  'convex': 81.03,
  'invex': 88.85,
  'ordinary': 85.89},
 '100': {'l_0.5': 78.27,
  'l_1': 82.07,
  'l_2': 86.41,
  'l_inf': 87.02,
  'stereo': 87.99,
  'linear': 88.67,
  'convex': 78.07,
  'invex': 87.85,
  'ordinary': 87.25},
 '500': {'l_0.5': 79.94,
  'l_1': 83.43,
  'l_2': 87.98,
  'l_inf': 88.23,
  'stereo': 89.26,
  'linear': 89.62,
  'convex': 79.68,
  'invex': 88.69,
  'ordinary': 87.14}}

In [27]:
xx

tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., 

In [28]:
# net[0].centers.shape

In [29]:
# exp_acc_vals = \
# {'5': {'l_0.5': 67.05,
#   'l_1': 70.7,
#   'l_2': 78.15,
#   'l_inf': 79.52,
#   'stereo': 82.19,
#   'linear': 82.74,
#   'convex': 79.49,
#   'invex': 88.26,
#   'ordinary': 83.55},
#  '10': {'l_0.5': 72.08,
#   'l_1': 77.91,
#   'l_2': 82.35,
#   'l_inf': 83.98,
#   'stereo': 84.73,
#   'linear': 84.89,
#   'convex': 78.99,
#   'invex': 88.41,
#   'ordinary': 81.69}}

In [30]:
# data_file = "./outputs/00.2_exp_acc_dict.json"
# with open(data_file, "w") as f:
#     json.dump(exp_acc_vals, f, indent=3)

In [31]:
# # Opening JSON file
# with open(data_file, 'r') as f:
#     exp_acc_vals = json.load(f)

In [32]:
exp_acc_vals

{'5': {'l_0.5': 67.05,
  'l_1': 70.7,
  'l_2': 78.15,
  'l_inf': 79.52,
  'stereo': 82.19,
  'linear': 82.74,
  'convex': 79.49,
  'invex': 88.26,
  'ordinary': 83.26},
 '10': {'l_0.5': 72.08,
  'l_1': 77.91,
  'l_2': 82.35,
  'l_inf': 83.98,
  'stereo': 84.73,
  'linear': 84.89,
  'convex': 78.99,
  'invex': 88.41,
  'ordinary': 86.11},
 '20': {'l_0.5': 75.54,
  'l_1': 80.08,
  'l_2': 83.82,
  'l_inf': 85.17,
  'stereo': 85.86,
  'linear': 86.27,
  'convex': 81.03,
  'invex': 88.85,
  'ordinary': 85.89},
 '100': {'l_0.5': 78.27,
  'l_1': 82.07,
  'l_2': 86.41,
  'l_inf': 87.02,
  'stereo': 87.99,
  'linear': 88.67,
  'convex': 78.07,
  'invex': 87.85,
  'ordinary': 87.25},
 '500': {'l_0.5': 79.94,
  'l_1': 83.43,
  'l_2': 87.98,
  'l_inf': 88.23,
  'stereo': 89.26,
  'linear': 89.62,
  'convex': 79.68,
  'invex': 88.69,
  'ordinary': 87.14}}